In [1]:
import cv2 
from cv2imshow.cv2imshow import cv2_imshow_single
IMAGE_FILE = 'maxresdefault.jpg'

img = cv2.imread(IMAGE_FILE)
window_name = 'Image'
cv2.imshow(window_name, img)
cv2.waitKey(0)

-1

In [2]:
pip install mediapipe 

In [4]:
# STEP 1: Import the necessary modules.
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import urllib.request

def visualize(image: np.ndarray, detection_result) -> np.ndarray:
    """
    Draw bounding boxes and labels from MediaPipe ObjectDetector on an image.
    """
    for detection in detection_result.detections:
        # Bounding box
        bbox = detection.bounding_box
        start_point = (int(bbox.origin_x), int(bbox.origin_y))
        end_point = (int(bbox.origin_x + bbox.width), int(bbox.origin_y + bbox.height))
        cv2.rectangle(image, start_point, end_point, (0, 255, 0), 2)

        # Label and score
        if detection.categories:
            category = detection.categories[0]
            label = f"{category.category_name}: {category.score:.2f}"
            cv2.putText(image, label, (start_point[0], start_point[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
    return image

# Download the model if it does not exist
model_path = 'efficientdet.tflite'
if not os.path.exists(model_path):
    url = 'https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/int8/1/efficientdet_lite0.tflite'
    urllib.request.urlretrieve(url, model_path)

# STEP 2: Create an ObjectDetector object.
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.ObjectDetectorOptions(base_options=base_options,
                                       score_threshold=0.5)
detector = vision.ObjectDetector.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file(IMAGE_FILE)

# STEP 4: Detect objects in the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
image_copy = np.copy(image.numpy_view())
annotated_image = visualize(image_copy, detection_result)
rgb_annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
window_name = 'Image'
cv2.imshow(window_name, rgb_annotated_image)
cv2.waitKey(0)

-1

In [5]:
import cv2
import mediapipe as mp


In [6]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Create Pose object
pose = mp_pose.Pose(static_image_mode=True,       # True for image, False for video stream
                    model_complexity=2,          # 0,1,2 → higher is more accurate
                    enable_segmentation=False,
                    min_detection_confidence=0.5)



In [7]:
image = cv2.imread("maxresdefault.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # MediaPipe expects RGB


In [8]:
results = pose.process(image_rgb)


In [9]:
if results.pose_landmarks:
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_pose.POSE_CONNECTIONS,     # Draw connections between joints
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
    )


In [10]:
cv2.imshow("Pose Landmarks", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [15]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=2,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# Input and output video paths
input_video_path = "male_bad_discus.mov"
output_video_path = "output_pose2.mp4"

# Open video
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    raise IOError(f"Cannot open video {input_video_path}")

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer to save annotated video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

reference_landmarks = []  # list of frames
user_landmarks = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process frame
    results = pose.process(image_rgb)

    # Draw landmarks if detected
    if results.pose_landmarks:
        frame_landmarks = []
        for lm in results.pose_landmarks.landmark:
            frame_landmarks.append((lm.x, lm.y, lm.z))  # normalized coords
        reference_landmarks.append(frame_landmarks)
        
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
        )

    # Write annotated frame to output
    out.write(frame)

# Release resources
cap.release()
out.release()
pose.close()
cv2.destroyAllWindows()

print(f"Annotated video saved to {output_video_path}")


Annotated video saved to output_pose2.mp4


In [16]:
import numpy as np

def angle_between_points(a, b, c):
    """
    Computes angle at point b (in degrees) formed by points a-b-c
    """
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cos_angle, -1.0, 1.0)))
    return angle

In [17]:
differences = []
for ref_frame, user_frame in zip(reference_angles, user_angles):
    frame_diff = np.abs(np.array(ref_frame) - np.array(user_frame))
    differences.append(frame_diff)


NameError: name 'reference_angles' is not defined

In [22]:
import cv2
import mediapipe as mp
import numpy as np

# -----------------------------
# Helper functions
# -----------------------------
def extract_landmarks(video_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False,
                        model_complexity=2,
                        enable_segmentation=False,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    frames = []
    landmarks_seq = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            landmarks_seq.append(results.pose_landmarks)
        else:
            landmarks_seq.append(None)  # placeholder

    cap.release()
    pose.close()
    return frames, landmarks_seq

def draw_landmarks_on_frame(frame, landmarks, mp_drawing, mp_pose):
    if landmarks:
        mp_drawing.draw_landmarks(
            frame,
            landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
        )
    return frame

# -----------------------------
# Main script
# -----------------------------
if __name__ == "__main__":
    reference_video = "Women_final_discus.mp4"
    user_video = "Male_final_discus.mp4"
    output_video_path = "comparison_overlay.mp4"

    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    # Extract frames and landmarks for both videos
    print("Processing reference video...")
    ref_frames, ref_landmarks = extract_landmarks(reference_video)
    print("Processing user video...")
    user_frames, user_landmarks = extract_landmarks(user_video)

    # Make sure both videos have the same number of frames
    num_frames = min(len(ref_frames), len(user_frames))
    width = max(ref_frames[0].shape[1], user_frames[0].shape[1])
    height = max(ref_frames[0].shape[0], user_frames[0].shape[0])

    # Create VideoWriter for output
    fps = 30  # adjust to your video FPS if needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width*2, height))

    print("Creating comparison video...")
    for i in range(num_frames):
        ref_frame = ref_frames[i].copy()
        user_frame = user_frames[i].copy()

        # Draw landmarks
        ref_frame = draw_landmarks_on_frame(ref_frame, ref_landmarks[i], mp_drawing, mp_pose)
        user_frame = draw_landmarks_on_frame(user_frame, user_landmarks[i], mp_drawing, mp_pose)

        # Resize frames to same height if necessary
        ref_frame = cv2.resize(ref_frame, (width, height))
        user_frame = cv2.resize(user_frame, (width, height))

        # Concatenate side by side
        combined_frame = np.hstack((ref_frame, user_frame))

        # Write to output video
        out.write(combined_frame)

    out.release()
    print(f"Comparison video saved to {output_video_path}")


Processing reference video...
Processing user video...
Creating comparison video...
Comparison video saved to comparison_overlay.mp4


In [17]:
import cv2
import mediapipe as mp
import numpy as np

# -----------------------------
# Helper functions
# -----------------------------
def extract_landmarks(video_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False,
                        model_complexity=2,
                        enable_segmentation=False,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    frames = []
    landmarks_seq = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            landmarks_seq.append(results.pose_landmarks)
        else:
            landmarks_seq.append(None)

    cap.release()
    pose.close()
    return frames, landmarks_seq

def launch_angle_vector(landmarks, frame_idx, shoulder_side='RIGHT', image_shape=None):
    mp_pose = mp.solutions.pose
    lm = landmarks[frame_idx]
    if lm is None:
        return None, None, None

    if shoulder_side=='RIGHT':
        shoulder = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        elbow = lm.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        wrist = lm.landmark[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    else:
        shoulder = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        elbow = lm.landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        wrist = lm.landmark[mp_pose.PoseLandmark.LEFT_WRIST.value]

    dx = wrist.x - elbow.x
    dy = elbow.y - wrist.y  # flip y for image coordinates
    angle_rad = np.arctan2(dy, dx)
    angle_deg = np.degrees(angle_rad)

    elbow_px = wrist_px = shoulder_px = None
    if image_shape:
        h, w, _ = image_shape
        elbow_px = (int(elbow.x*w), int(elbow.y*h))
        wrist_px = (int(wrist.x*w), int(wrist.y*h))
        shoulder_px = (int(shoulder.x*w), int(shoulder.y*h))

    return angle_deg, elbow_px, wrist_px

def torso_rotation_speed(landmarks_seq, fps):
    speeds = []
    for i in range(1, len(landmarks_seq)):
        lm_prev = landmarks_seq[i-1]
        lm_curr = landmarks_seq[i]
        if lm_prev is None or lm_curr is None:
            speeds.append(0)
            continue
        left_prev = lm_prev.landmark[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value]
        right_prev = lm_prev.landmark[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_curr = lm_curr.landmark[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value]
        right_curr = lm_curr.landmark[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value]

        angle_prev = np.degrees(np.arctan2(left_prev.y - right_prev.y, left_prev.x - right_prev.x))
        angle_curr = np.degrees(np.arctan2(left_curr.y - right_curr.y, left_curr.x - right_curr.x))
        speed = (angle_curr - angle_prev) * fps
        speeds.append(speed)
    return speeds


def detect_throw_circle(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9,9), 2)

    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100,
                               param1=50, param2=30, minRadius=50, maxRadius=500)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        x, y, r = circles[0][0]
        return (x, y), r
    else:
        h, w = frame.shape[:2]
        return (w//2, h//2), min(w,h)//3

# -----------------------------
# Main script
# -----------------------------
if __name__ == "__main__":
    video_path = "Male_final_discus.mp4"  # replace with your video path
    shoulder_side = 'RIGHT'        # change to 'LEFT' if left-handed throw
    output_video_path = "annotated_throw_skeleton.mp4"

    print("Extracting landmarks...")
    frames, landmarks_seq = extract_landmarks(video_path)
    num_frames = len(frames)
    h, w, _ = frames[0].shape

    # Detect throwing circle
    circle_center, circle_radius = detect_throw_circle(frames[0])
    print(f"Detected circle center: {circle_center}, radius: {circle_radius}")

    # Let user select release frame
    print("Press 's' to select the release frame while watching the video.")
    release_frame = None
    for idx, frame in enumerate(frames):
        display_frame = frame.copy()
        cv2.putText(display_frame, f"Frame {idx}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.imshow("Select Release Frame", display_frame)
        key = cv2.waitKey(30) & 0xFF
        if key == ord('s'):
            release_frame = idx
            break
        elif key == ord('q'):
            break
    cv2.destroyAllWindows()

    if release_frame is None:
        print("No release frame selected. Exiting.")
        exit()

    # Launch angle
    angle_deg, elbow_px, wrist_px = launch_angle_vector(landmarks_seq, release_frame, shoulder_side, frames[0].shape)
    print(f"Selected release frame: {release_frame}")
    print(f"Launch angle at release: {angle_deg:.2f} degrees")

    # Torso rotation speed
    fps = 30
    rotation_speeds = torso_rotation_speed(landmarks_seq, fps)
    rotation_at_release = rotation_speeds[release_frame-1] if release_frame > 0 else 0
    print(f"Torso rotation speed at release: {rotation_at_release:.2f} deg/sec")

    # MediaPipe drawing utils
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    # Create short clip around release
    clip_range = 15
    start = max(release_frame - clip_range, 0)
    end = min(release_frame + clip_range, num_frames)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 30, (w, h))

    for i in range(start, end):
        frame = frames[i].copy()

        # Draw skeleton
        if landmarks_seq[i]:
            mp_drawing.draw_landmarks(frame, landmarks_seq[i], mp_pose.POSE_CONNECTIONS)


        # Draw feet positions
        lm = landmarks_seq[i]
        if lm:
            lf = lm.landmark[mp.solutions.pose.PoseLandmark.LEFT_FOOT_INDEX.value]
            rf = lm.landmark[mp.solutions.pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
            lf_px = (int(lf.x*w), int(lf.y*h))
            rf_px = (int(rf.x*w), int(rf.y*h))
            color =  (0,0,255)
            cv2.circle(frame, lf_px, 5, color, -1)
            cv2.circle(frame, rf_px, 5, color, -1)

        # Draw launch angle arrow at release
        if i == release_frame and elbow_px and wrist_px:
            cv2.arrowedLine(frame, elbow_px, wrist_px, (0,0,255), 4, tipLength=0.2)
            cv2.putText(frame, f"{angle_deg:.1f} deg", (elbow_px[0], elbow_px[1]-20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.putText(frame, f"Rot speed: {rotation_at_release:.1f} deg/s", (10,60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)

        out.write(frame)

    out.release()
    print(f"Annotated video saved to {output_video_path}")


Extracting landmarks...
Detected circle center: (357, 20), radius: 277
Press 's' to select the release frame while watching the video.
Selected release frame: 52
Launch angle at release: 10.70 degrees
Torso rotation speed at release: 135.45 deg/sec
Annotated video saved to annotated_throw_skeleton.mp4
